<img src="Imagenes/Mac_wallpaper_3.png" width="50%">

In [ ]:
!pip install "qiskit[visualization]" --user

In [ ]:
!pip install qiskit-aer --user

# Teletransportación Cuántica

Si bien el nombre "teletransportación" nos hace pensar casi de de inmediato en transmitir materia de un lugar a otro de manera instantánea, la teletransportación cuántica es más bien un método de "pasar" el estado de un qubit a otro, hablando a grandes rasgos.

### Escenario

Supongamos que Alice quiere enviarle el estado de un qubit a Bob. Esto puede parecer trivial si el qubit de Alice se encuentra en alguno de los estados base, pues todo lo que tiene que hacer es decirle a Bob en qué estado se encuentra su qubit. Es por ello que vamos a tomar en cuenta que el qubit de Alice se encuentra en el estado más general de un qubit:

$\hspace{9 cm} |A\rangle = a|0\rangle + b|1\rangle \hspace{0.5cm}$ donde $\hspace{0.5 cm} a,b \in \mathbb{C}$

Si Alice quiere enviarle el estado del qubit $|A\rangle$ tal cual a Bob, buscando perder la menor cantidad posible de información, deberá hacer varias copias de su qubit y medirlos todos para aproximar el valor de $a$ y $b$ usando las estadísticas de sus mediciones. La cantidad de copias que haga de su qubit dependerá de qué tan precisa quiere que sea su "medición" de $a$ y $b$.

Es aquí donde el protocolo de teletransportación cuántica le permite a Alice mandarle el estado de su qubit a un qubit que Bob posea, al costo de "destruir" su propio qubit. Es decir, al final de este protocolo Bob tendrá un qubit con el mismo estado que el qubit que Alice quería enviarle, pero su qubit ya no se encontrará en el estado inicial. Esto se consigue por medio del entrelazamiento.

### Paso 1: Estado inicial

Para comenzar, necesitamos de tres qubits: el qubit que Alice quiere enviarle a Bob y un par de qubits entrelazados. Bob y Alice tendrán un qubit cada uno de dicho par.

Entonces, $q_{0}$ y $q_{1}$ serán el par de qubits entrelazados y $q_{2}$ el qubit que Alice quiere enviar. Al igual que en el caso de codificación superdensa, empezaremos entrelazando un par de qubits utilizando uno de los estados de Bell:

$\hspace{11 cm} \dfrac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$

Por lo que el sistema completo (con los tres qubits) se encuentra inicialmente en el estado

$\hspace{5 cm} (a|0\rangle + b|1\rangle)\otimes\left(\dfrac{1}{\sqrt{2}}|00\rangle + \dfrac{1}{\sqrt{2}}|11\rangle\right) = \dfrac{1}{\sqrt{2}}(a|000\rangle + a|011\rangle + b|100\rangle + b|111\rangle)$

In [ ]:
from qiskit import QuantumCircuit, execute, Aer

qc = QuantumCircuit(3)

qc.h(0)
qc.cx(0,1)

qc.draw(output="mpl")

### Paso 2: Operaciones de Alice

Ya con nuestro estado inicial, le damos el qubit $q_{1}$ a Alice y el qubit $q_{0}$ a Bob. Ya con sus dos qubits, Alice debe realizar dos operaciones: la primera es aplicar una compuerta CNOT, tomando al qubit $q_{2}$ como el control y al qubit $q_{1}$ como el objetivo. De esta manera, el nuevo estado del sistema es

$\hspace{8 cm} \dfrac{1}{\sqrt{2}}(a|000\rangle + a|011\rangle + b|110\rangle + b|101\rangle)$

Luego, debe aplicar una compuerta Hadamard a $q_{2}$. De esta manera, el estado del sistema se convierte en

$\dfrac{1}{2}(a|000\rangle + a|100\rangle + a|011\rangle + a|111\rangle + b|010\rangle - b|110\rangle + b|001\rangle - b|101\rangle) = \hspace{10 cm} \dfrac{1}{2}[|00\rangle(a|0\rangle + b|1\rangle)+|01\rangle(a|1\rangle + b|0\rangle)+|10\rangle(a|0\rangle-b|1\rangle)+|11\rangle(a|1\rangle-b|0\rangle)]$

Ahora, todo lo que resta por hacer en el lado de Alice es medir sus dos qubits y mandarle a Bob los dos bits que obtendrá como resultado de su medición. 

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister

#Declaramos nuestros qubits y bits por separado, para que sea más fácil identificarlos dentro de los métodos
q = QuantumRegister(3)
c = ClassicalRegister(2)

qc = QuantumCircuit(q,c)

qc.h(q[0])
qc.cx(q[0],q[1])
qc.barrier()

qc.cx(q[2],q[1])
qc.h(q[2])
qc.measure(q[2],c[1])
qc.measure(q[1],c[0])

qc.draw(output="mpl")

### Paso 3: Procesamiento de Bob

Al momento en que Alice mida sus dos qubits, el estado del sistema completo colapsará a una de cuatro posibilidades. Entonces, al momento en que Bob reciba los dos bits de Alice (resultado de su medición) su qubit se encontrará en uno de cuatro posibles estados:

| Medición de Alice | Qubit de Bob |
| :---: | :---: |
| 00 | $a|0\rangle + b|1\rangle$ |
| 01 | $a|1\rangle + b|0\rangle$ |
| 10 | $a|0\rangle - b|1\rangle$ |
| 11 | $a|1\rangle - b|0\rangle$ |

De esta forma, vemos que para que el qubit de Bob termine en el mismo estado que el qubit que Alice quiere enviarle, este debe hacer las siguientes operaciones:

- Si Alice manda 00 $\rightarrow$ Bob no hace nada
- Si Alice manda 01 $\rightarrow$ Bob aplica una compuerta X
- Si Alice manda 10 $\rightarrow$ Bob aplica una compuerta Z
- Si Alice manda 11 $\rightarrow$ Bob aplica primero una compuerta X y luego una compuerta Z

De esta forma vemos que, sin importar cuál de los cuatro estados mida Alice (cada uno tiene 25% de probabilidades de ser medido), Bob siempre terminará con el estado que ella le quería enviar.

Para aplicar condicionales sobre nuestro circuito podemos usar el métod `.c_if(bit,valor)` donde *bit* es el nombre del bit que queremos revisar y *valor* es el valor binario que estamos buscando. <br>
Por ejemplo, si escribo la línea `qc.x(q[0]).c_if(c[0],1)` le estoy indicando al programa que aplique una compuerta $X$ al qubit $q_{0}$ si el bit $0$ tiene el valor $1$. Esto solo se puede utilizar con los bits clásicos, por lo que es necesario hacer una medición antes de poder emplearlo.

In [ ]:
q = QuantumRegister(3)
c = ClassicalRegister(2)

qc = QuantumCircuit(q,c)

qc.h(q[0])
qc.cx(q[0],q[1])
qc.barrier()

qc.cx(q[2],q[1])
qc.h(q[2])
qc.measure(q[2],c[1])
qc.measure(q[1],c[0])
qc.barrier()

#Aplicamos las compuertas según sea el caso
qc.x(q[0]).c_if(c[0],1)
qc.z(q[0]).c_if(c[1],1)

qc.draw(output="mpl")

### Utilizar el protocolo completo

Ahora que ya tenemos el protocolo completo, vamos a ponerlo a prueba. Para ello, vamos a inicializar el qubit de Alice en un estado aleatorio y vamos a utilizar la esfera de Bloch para asegurarnos de que el estado de su qubit se transportó correctamente al qubit de Bob.

In [ ]:
from qiskit.visualization import plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import random_statevector

#Generamos un vector de estado aleatorio
psi = random_statevector(2)

#Imprimimos su vector de estado utilizando LATEX, para que sea más sencillo leerlo
display(array_to_latex(psi, prefix="|\\psi\\rangle ="))

#Graficamos el estado en la esfera de Bloch
plot_bloch_multivector(psi)

In [ ]:
q = QuantumRegister(3)
c = ClassicalRegister(2)

qc = QuantumCircuit(q,c)

#Inicializamos el qubit de Alice en el estado |psi> que generamos
qc.initialize(psi,q[2])

#Aplicamos el protocolo de teletransportación
qc.h(q[0])
qc.cx(q[0],q[1])
qc.barrier()

qc.cx(q[2],q[1])
qc.h(q[2])
qc.measure(q[2],c[1])
qc.measure(q[1],c[0])
qc.barrier()

qc.x(q[0]).c_if(c[0],1)
qc.z(q[0]).c_if(c[1],1)

In [ ]:
job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_statevector(qc,precision).data
#Graficamos el estado de los tres qubits en tres esferas de Bloch distintas, una para cada qubit
#Utilizamos el método reverse_bits=True para que las esferas de Bloch se muestren utilizando el mismo orden que Qiskit
plot_bloch_multivector(vector_estado,reverse_bits=True)

Como puedes ver, los dos qubits de Alice terminan en uno de los dos estados base mientras que el qubit de Bob se encuentra en el estado que Alice quería enviarle.

Este protocolo es similar al de codificación superdensa, con la diferencia de que mientras en ese protocolo usábamos un qubit para transmitir dos bits de información, aquí utilizamos dos bits de información para transmitir el estado de un qubit.